<a href="https://colab.research.google.com/github/dfatpnuk/bayesian-rl/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pgmpy
import numpy as np

In [ ]:
from pgmpy.models import BayesianNetwork
G = BayesianNetwork()

In [ ]:
G.add_nodes_from(['a', 'b'])
G.add_edges_from([('a', 'b'), ('b', 'c')])

In [ ]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete.CPD import TabularCPD
student = BayesianNetwork([('diff', 'grades'), ('aptitude', 'grades')])
grades_cpd = TabularCPD('grades', 3, [[0.1,0.1,0.1,0.1,0.1,0.1],
                                      [0.1,0.1,0.1,0.1,0.1,0.1],
                                      [0.8,0.8,0.8,0.8,0.8,0.8]],
                        evidence=['diff', 'aptitude'], evidence_card=[2, 3],
                        state_names={'grades': ['gradeA', 'gradeB', 'gradeC'],
                                     'diff': ['easy', 'hard'],
                                     'aptitude': ['low', 'medium', 'high']})
student.add_cpds(grades_cpd)

In [ ]:
from pgmpy.models import BayesianNetwork
G = BayesianNetwork()
G.add_nodes_from(['grade', 'intel'])
G.add_edge('grade', 'intel')

In [ ]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
model = BayesianNetwork([('A', 'B'), ('B', 'C')])
cpd_a = TabularCPD('A', 2, [[0.2], [0.8]])
cpd_b = TabularCPD('B', 2, [[0.3, 0.7], [0.7, 0.3]],
                   evidence=['A'],
                   evidence_card=[2])
cpd_c = TabularCPD('C', 2, [[0.1, 0.9], [0.9, 0.1]],
                   evidence=['B'],
                   evidence_card=[2])
model.add_cpds(cpd_a, cpd_b, cpd_c)
copy_model = model.copy()
copy_model.nodes()
copy_model.edges()
len(copy_model.get_cpds())

3

In [ ]:
from pgmpy.utils import get_example_model
asia = get_example_model('asia')
asia.edges()
do_bronc = asia.do(['bronc'])

In [ ]:
asia.edges()

OutEdgeView([('asia', 'tub'), ('tub', 'either'), ('smoke', 'lung'), ('smoke', 'bronc'), ('lung', 'either'), ('bronc', 'dysp'), ('either', 'xray'), ('either', 'dysp')])

In [ ]:
import pandas as pd
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import MaximumLikelihoodEstimator
data = pd.DataFrame(data={'A': [0, 0, 1], 'B': [0, 1, 0], 'C': [1, 1, 0]})
model = BayesianNetwork([('A', 'C'), ('B', 'C')])
model.fit(data)
model.get_cpds()

[<TabularCPD representing P(A:2) at 0x78e04e1afeb0>,
 <TabularCPD representing P(C:2 | A:2, B:2) at 0x78e04e20d8d0>,
 <TabularCPD representing P(B:2) at 0x78e04e20c040>]

Examples taken from: https://pgmpy.org/models/bayesiannetwork.html

In [ ]:
from pgmpy.utils import get_example_model
from pgmpy.sampling import BayesianModelSampling
model = get_example_model('alarm')
# Generate some new data.
data = BayesianModelSampling(model).forward_sample(int(1e3))
model.fit_update(data)

  0%|          | 0/37 [00:00<?, ?it/s]

Can we apply Reinforcement Learning to Bayesian Network Structure Learning?

Can we apply a greedy hill-climbing approach to this task?

Structure Learning Example https://pgmpy.org/examples/Structure%20Learning%20in%20Bayesian%20Networks.html

In [ ]:
from itertools import combinations

import networkx as nx
from sklearn.metrics import f1_score

from pgmpy.estimators import PC, HillClimbSearch, ExhaustiveSearch
from pgmpy.estimators import K2Score
from pgmpy.utils import get_example_model
from pgmpy.sampling import BayesianModelSampling

In [ ]:
model = get_example_model("alarm")
samples = BayesianModelSampling(model).forward_sample(size=int(1e3))
samples.head()

  0%|          | 0/37 [00:00<?, ?it/s]

,HISTORY,CVP,PCWP,HYPOVOLEMIA,LVEDVOLUME,LVFAILURE,STROKEVOLUME,ERRLOWOUTPUT,HRBP,HREKG,...,MINVOLSET,VENTMACH,VENTTUBE,VENTLUNG,VENTALV,ARTCO2,CATECHOL,HR,CO,BP
0,FALSE,NORMAL,HIGH,TRUE,HIGH,FALSE,NORMAL,FALSE,HIGH,HIGH,...,NORMAL,NORMAL,LOW,ZERO,ZERO,HIGH,HIGH,HIGH,NORMAL,LOW
1,FALSE,HIGH,HIGH,FALSE,HIGH,FALSE,HIGH,FALSE,HIGH,HIGH,...,NORMAL,NORMAL,LOW,ZERO,ZERO,HIGH,HIGH,HIGH,HIGH,HIGH
2,FALSE,NORMAL,NORMAL,FALSE,NORMAL,FALSE,NORMAL,TRUE,NORMAL,HIGH,...,NORMAL,NORMAL,LOW,ZERO,ZERO,HIGH,HIGH,HIGH,HIGH,LOW
3,FALSE,NORMAL,NORMAL,FALSE,NORMAL,FALSE,NORMAL,FALSE,HIGH,HIGH,...,NORMAL,NORMAL,LOW,LOW,HIGH,LOW,HIGH,HIGH,HIGH,HIGH
4,FALSE,NORMAL,HIGH,TRUE,HIGH,FALSE,LOW,FALSE,HIGH,HIGH,...,NORMAL,NORMAL,LOW,ZERO,ZERO,HIGH,HIGH,HIGH,NORMAL,LOW


In [ ]:
# Funtion to evaluate the learned model structures.
def get_f1_score(estimated_model, true_model):
    nodes = estimated_model.nodes()
    est_adj = nx.to_numpy_array(
        estimated_model.to_undirected(), nodelist=nodes, weight=None
    )
    true_adj = nx.to_numpy_array(
        true_model.to_undirected(), nodelist=nodes, weight=None
    )

    f1 = f1_score(np.ravel(true_adj), np.ravel(est_adj))
    print("F1-score for the model skeleton: ", f1)

In [ ]:
#Learning the structure using Hill Climbing
scoring_method = K2Score(data=samples)  # K2 Scoring
est = HillClimbSearch(data=samples)
estimated_model = est.estimate(
    scoring_method=scoring_method, max_indegree=4, max_iter=int(1e4) # <- Learned Model Structure
)
get_f1_score(estimated_model, model) # <- Evaluate the learned model structure.

  0%|          | 0/10000 [00:00<?, ?it/s]

F1-score for the model skeleton:  0.6935483870967742


Structure Learning Example: https://pgmpy.org/detailed_notebooks/10.%20Learning%20Bayesian%20Networks%20from%20Data.html

In [ ]:
import pandas as pd
data = pd.DataFrame(data={'fruit': ["banana", "apple", "banana", "apple", "banana","apple", "banana",
                                    "apple", "apple", "apple", "banana", "banana", "apple", "banana",],
                          'tasty': ["yes", "no", "yes", "yes", "yes", "yes", "yes",
                                    "yes", "yes", "yes", "yes", "no", "no", "no"],
                          'size': ["large", "large", "large", "small", "large", "large", "large",
                                    "small", "large", "large", "large", "large", "small", "small"]})
print(data)

     fruit tasty   size
0   banana   yes  large
1    apple    no  large
2   banana   yes  large
3    apple   yes  small
4   banana   yes  large
5    apple   yes  large
6   banana   yes  large
7    apple   yes  small
8    apple   yes  large
9    apple   yes  large
10  banana   yes  large
11  banana    no  large
12   apple    no  small
13  banana    no  small


In [ ]:
from pgmpy.models import BayesianModel

model = BayesianModel([('fruit', 'tasty'), ('size', 'tasty')])  # fruit -> tasty <- size

In [ ]:
from pgmpy.estimators import ParameterEstimator
pe = ParameterEstimator(model, data)
print("\n", pe.state_counts('fruit'))  # unconditional
print("\n", pe.state_counts('tasty'))  # conditional on fruit and size


         fruit
apple       7
banana      7

 fruit apple       banana      
size  large small  large small
tasty                         
no      1.0   1.0    1.0   1.0
yes     3.0   2.0    5.0   0.0


In [ ]:
# Maximum Likelihood Estimation (not suitable for distributions with high cardinality)
from pgmpy.estimators import MaximumLikelihoodEstimator
mle = MaximumLikelihoodEstimator(model, data)
print(mle.estimate_cpd('fruit'))  # unconditional
print(mle.estimate_cpd('tasty'))  # conditional

+---------------+-----+
| fruit(apple)  | 0.5 |
+---------------+-----+
| fruit(banana) | 0.5 |
+---------------+-----+
+------------+--------------+--------------------+---------------------+---------------+
| fruit      | fruit(apple) | fruit(apple)       | fruit(banana)       | fruit(banana) |
+------------+--------------+--------------------+---------------------+---------------+
| size       | size(large)  | size(small)        | size(large)         | size(small)   |
+------------+--------------+--------------------+---------------------+---------------+
| tasty(no)  | 0.25         | 0.3333333333333333 | 0.16666666666666666 | 1.0           |
+------------+--------------+--------------------+---------------------+---------------+
| tasty(yes) | 0.75         | 0.6666666666666666 | 0.8333333333333334  | 0.0           |
+------------+--------------+--------------------+---------------------+---------------+


In [ ]:
# Calibrate all CPDs of `model` using MLE:
model.fit(data, estimator=MaximumLikelihoodEstimator)

In [ ]:
from pgmpy.estimators import BayesianEstimator
est = BayesianEstimator(model, data)

print(est.estimate_cpd('tasty', prior_type='BDeu', equivalent_sample_size=10))

+------------+---------------------+--------------------+--------------------+---------------------+
| fruit      | fruit(apple)        | fruit(apple)       | fruit(banana)      | fruit(banana)       |
+------------+---------------------+--------------------+--------------------+---------------------+
| size       | size(large)         | size(small)        | size(large)        | size(small)         |
+------------+---------------------+--------------------+--------------------+---------------------+
| tasty(no)  | 0.34615384615384615 | 0.4090909090909091 | 0.2647058823529412 | 0.6428571428571429  |
+------------+---------------------+--------------------+--------------------+---------------------+
| tasty(yes) | 0.6538461538461539  | 0.5909090909090909 | 0.7352941176470589 | 0.35714285714285715 |
+------------+---------------------+--------------------+--------------------+---------------------+


In [ ]:
import numpy as np
import pandas as pd
from pgmpy.models import BayesianModel
from pgmpy.estimators import BayesianEstimator

# generate data
data = pd.DataFrame(np.random.randint(low=0, high=2, size=(5000, 4)), columns=['A', 'B', 'C', 'D'])
model = BayesianModel([('A', 'B'), ('A', 'C'), ('D', 'C'), ('B', 'D')])

model.fit(data, estimator=BayesianEstimator, prior_type="BDeu") # default equivalent_sample_size=5
for cpd in model.get_cpds():
    print(cpd)

+------+----------+
| A(0) | 0.499401 |
+------+----------+
| A(1) | 0.500599 |
+------+----------+
+------+------------------+--------------------+
| A    | A(0)             | A(1)               |
+------+------------------+--------------------+
| B(0) | 0.48499699939988 | 0.4882259030133706 |
+------+------------------+--------------------+
| B(1) | 0.51500300060012 | 0.5117740969866295 |
+------+------------------+--------------------+
+------+---------------------+--------------------+--------------------+---------------------+
| A    | A(0)                | A(0)               | A(1)               | A(1)                |
+------+---------------------+--------------------+--------------------+---------------------+
| D    | D(0)                | D(1)               | D(0)               | D(1)                |
+------+---------------------+--------------------+--------------------+---------------------+
| C(0) | 0.49569724232348916 | 0.5065506653019447 | 0.4929203539823009 | 0.4995948

To learn model structure (a DAG) from a data set, there are two broad techniques:

* score-based structure learning
* constraint-based structure learning


In [ ]:
#Scoring Functions
import pandas as pd
import numpy as np
from pgmpy.estimators import BDeuScore, K2Score, BicScore
from pgmpy.models import BayesianModel

# create random data sample with 3 variables, where Z is dependent on X, Y:
data = pd.DataFrame(np.random.randint(0, 4, size=(5000, 2)), columns=list('XY'))
data['Z'] = data['X'] + data['Y']

bdeu = BDeuScore(data, equivalent_sample_size=5)
k2 = K2Score(data)
bic = BicScore(data)

model1 = BayesianModel([('X', 'Z'), ('Y', 'Z')])  # X -> Z <- Y
model2 = BayesianModel([('X', 'Z'), ('X', 'Y')])  # Y <- X -> Z


print(bdeu.score(model1))
print(k2.score(model1))
print(bic.score(model1))

print(bdeu.score(model2))
print(k2.score(model2))
print(bic.score(model2))

-13940.773334801332
-14331.620074152623
-14296.811973611726
-20906.185120390204
-20933.017090678048
-20950.225089647756


In [ ]:
print(bdeu.local_score('Z', parents=[]))
print(bdeu.local_score('Z', parents=['X']))
print(bdeu.local_score('Z', parents=['X', 'Y']))

-9179.545622774995
-6993.0813396872945
-57.12197800235225


In [ ]:
#Search Strategies
from pgmpy.estimators import ExhaustiveSearch

es = ExhaustiveSearch(data, scoring_method=bic)
best_model = es.estimate()
print(best_model.edges())

print("\nAll DAGs by score:")
for score, dag in reversed(es.all_scores()):
    print(score, dag.edges())

[('X', 'Z'), ('Y', 'Z')]

All DAGs by score:
-14296.811973611726 [('X', 'Z'), ('Y', 'Z')]
-14330.361658914524 [('Y', 'X'), ('Z', 'X'), ('Z', 'Y')]
-14330.361658914524 [('Y', 'Z'), ('Y', 'X'), ('Z', 'X')]
-14330.361658914524 [('X', 'Z'), ('Y', 'Z'), ('Y', 'X')]
-14330.361658914524 [('X', 'Y'), ('Z', 'X'), ('Z', 'Y')]
-14330.361658914524 [('X', 'Y'), ('X', 'Z'), ('Z', 'Y')]
-14330.361658914524 [('X', 'Y'), ('X', 'Z'), ('Y', 'Z')]
-16485.093757271607 [('X', 'Y'), ('Z', 'Y')]
-16485.178775787695 [('Y', 'X'), ('Z', 'X')]
-18761.858287471783 [('Z', 'X'), ('Z', 'Y')]
-18761.858287471783 [('X', 'Z'), ('Z', 'Y')]
-18761.858287471787 [('Y', 'Z'), ('Z', 'X')]
-20916.59038582887 [('Z', 'Y')]
-20916.590385828873 [('Y', 'Z')]
-20916.67540434496 [('Z', 'X')]
-20916.67540434496 [('X', 'Z')]
-20950.140071131667 [('Y', 'X'), ('Z', 'Y')]
-20950.140071131667 [('Y', 'Z'), ('Y', 'X')]
-20950.140071131667 [('X', 'Y'), ('Y', 'Z')]
-20950.225089647756 [('X', 'Z'), ('Y', 'X')]
-20950.225089647756 [('X', 'Y'), (

In [ ]:
from pgmpy.estimators import HillClimbSearch

# create some data with dependencies
data = pd.DataFrame(np.random.randint(0, 3, size=(2500, 8)), columns=list('ABCDEFGH'))
data['A'] += data['B'] + data['C']
data['H'] = data['G'] - data['A']

hc = HillClimbSearch(data)
best_model = hc.estimate(scoring_method=BicScore(data))
print(best_model.edges())

  0%|          | 0/1000000 [00:00<?, ?it/s]

[('A', 'B'), ('A', 'C'), ('B', 'C'), ('G', 'A'), ('H', 'A'), ('H', 'G')]


In [ ]:
#(Conditional) Independence Tests
from pgmpy.estimators import PC
from pgmpy.estimators.CITests import chi_square

data = pd.DataFrame(np.random.randint(0, 3, size=(2500, 8)), columns=list('ABCDEFGH'))
data['A'] += data['B'] + data['C']
data['H'] = data['G'] - data['A']
data['E'] *= data['F']

print(chi_square(X='B', Y='H', Z=[], data=data, significance_level=0.05))          # dependent
print(chi_square(X='B', Y='E', Z=[], data=data, significance_level=0.05))          # independent
print(chi_square(X='B', Y='H', Z=['A'], data=data, significance_level=0.05))       # independent
print(chi_square(X='A', Y='G', Z=[], data=data, significance_level=0.05))          # independent
print(chi_square(X='A', Y='G', Z=['H'], data=data, significance_level=0.05))       # dependent

False
True
True
False
False


/usr/local/lib/python3.10/dist-packages/pgmpy/estimators/CITests.py:547: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for z_state, df in data.groupby(Z):
/usr/local/lib/python3.10/dist-packages/pgmpy/estimators/CITests.py:547: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for z_state, df in data.groupby(Z):
